#Importation des librairies

In [4]:
#pipenv install pour installer les dependences
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px


#Importation des données


##dataset

In [5]:
data = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/78348f03-a11c-4a6b-b8db-2acf4fee81b1", sep="|", low_memory=False)

##Les villes de france

In [1]:
# https://sql.sh/736-base-donnees-villes-francaises
villes_france = pd.read_csv("villes_france.csv", sep=",",low_memory=False)

NameError: name 'pd' is not defined

#Nettoyage des données


##Pour le dataset

In [3]:
#pour nettoyer les données on va supprimer les colonnes où il n'y a pas de valeurs
clean_data = data.dropna(axis = 1, how = "all")

#on va aussi supprimer les valeurs comme B/T/Q ou Code_coie car elle ne vont pas nous servir
clean_data = data.drop(columns=['Identifiant de document', 'Reference document', '1 Articles CGI','2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',"B/T/Q","No disposition", "Code voie", "Prefixe de section", "Section", "No plan", "No Volume", "Code type local"])

#on doit aussi supprimer les lignes vides si il y en a 
clean_data = clean_data.dropna(axis = 0, how = "all")

# le nettoyage des données est terminé 
#Par la suite, nous allons utiliser le dataframe clean_data



NameError: name 'data' is not defined

##Pour les villes

In [ ]:
positions = villes_france.drop(columns=["Slug","Nom simple","Nom reel","Nom soundex","Nom metaphone","Canton","Population 2010","Population 1999","Densite 2010","Surface","Superficie"])

: 

##Ventes totales en 2023

In [ ]:
ventes_totales = clean_data
ventes_totales["Date mutation"] = pd.to_datetime(ventes_totales["Date mutation"], format='%d/%m/%Y')
ventes_par_date = ventes_totales.groupby("Date mutation").size().reset_index(name="Nombre de ventes")
fig = px.bar(ventes_par_date, x='Date mutation', y='Nombre de ventes', title='Nombre de ventes par date')
fig.show()

: 

In [ ]:
ventes_totales = clean_data
ventes_par_departement = ventes_totales.groupby('Code departement').size().reset_index(name="Nombre de ventes")
fig = px.bar(ventes_par_departement, x='Code departement', y='Nombre de ventes', title='Nombre de ventes par departement')
fig.show()

: 

In [ ]:
positions = positions.rename(columns={"Nom" : "Commune"})
ventes_par_commune = ventes_totales.groupby('Commune').size().reset_index(name="Nombre de ventes")
#ventes_paris = {"Nom" : "PARIS" , "Nombre de ventes" : }
#ventes_par_commune = ventes_par_commune.append()
villes_et_positions = pd.merge(positions, ventes_par_commune, on="Commune")
fig = px.density_mapbox(villes_et_positions, lat = "Latitude", lon = "Longitude", z='Nombre de ventes', radius=20, center=dict(lat=46.6035, lon=1.888334), zoom=3, mapbox_style = "open-street-map", title = 'Carte de densité des ventes immobilières en France')
fig.show()

: 

In [ ]:
ventes_par_commune[ventes_par_commune["Commune"] == "LYON 1er"]

: 

Pour mettre le code en HTML : jupyter nbconvert VotreNotebook.ipynb --to html


In [ ]:
clean_data.columns

: 